# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

weather_metadata = ("../output_data/my_weather_data.csv")

# Read the weather data and the study results
my_weather_data = pd.read_csv(weather_metadata)

# Combine the data into a single dataset
weather_df = pd.DataFrame(my_weather_data)
weather_df.head()


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,San Luis,-33.2950,-66.3356,57.83,26,9,17.87,AR,1663232400
1,1,Souillac,-20.5167,57.5167,76.44,62,15,17.60,MU,1663232400
2,2,Prabumulih,-3.4500,104.2500,90.09,48,39,3.18,ID,1663232400
3,3,Broome,42.2506,-75.8330,53.40,88,28,4.41,US,1663232400
4,4,Busselton,-33.6500,115.3333,60.64,68,62,14.99,AU,1663232400


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps.
gmaps.configure(api_key=g_key)



In [16]:
locations = weather_df[["Latitude", "Longitude"]].astype(float)

hum_rate = weather_df["Humidity"].astype(float)

fig = gmaps.figure(center=(48.0, 0.0), zoom_level=4)

marker_layer = gmaps.marker_layer(locations, #weights=hum_rate, dissipating=False 
                                 )

fig.add_layer(marker_layer)

plt.savefig("vacationheatmap.png")
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
first = weather_df.loc[weather_df["Max Temp"]<27,:]
second = first.loc[weather_df["Max Temp"]>21,:]
second.head()

#Wind speed less than 10 mph.
third = second.loc[second["Wind Speed"]<4.5,:]
third

#Zero cloudiness.
fourth = third.loc[third["Cloudiness"]==0,:]
fourth.head()


,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
62,62,Kokstad,-30.5472,29.4241,79.79,30,0,7.00,ZA,1663232400
121,121,Springbok,-29.6643,17.8865,78.78,8,0,1.16,ZA,1663232400
180,180,Darnah,32.7670,22.6367,78.01,56,0,8.55,LY,1663232400
248,248,Yarqā,31.9785,35.6947,77.41,52,0,4.32,JO,1663232400
320,320,Piet Retief,-27.0071,30.8132,79.63,39,0,4.74,ZA,1663232400


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
hotel_df = pd.DataFrame()
fifth = pd.DataFrame(fourth)
hotel_df['City'] = fifth['City']
hotel_df['Country'] = fifth['Country']
hotel_df['Lat'] = fifth['Latitude']
hotel_df['Lng'] = fifth['Longitude']
hotel_df['Hotel Name'] = ""
hotel_df.head()


,City,Country,Lat,Lng,Hotel Name
62,Kokstad,ZA,-30.5472,29.4241,
121,Springbok,ZA,-29.6643,17.8865,
180,Darnah,LY,32.7670,22.6367,
248,Yarqā,JO,31.9785,35.6947,
320,Piet Retief,ZA,-27.0071,30.8132,


In [19]:
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 62: Kokstad.
Closest hotel to Kokstad at -30.5472 , 29.4241 is Rocky Ridge Spur Steak Ranch.
------------
Retrieving Results for Index 121: Springbok.
Closest hotel to Springbok at -29.6643 , 17.8865 is Desert Rose Guest House.
------------
Retrieving Results for Index 180: Darnah.
Closest hotel to Darnah at 32.767 , 22.6367 is Al Banon for children clothes.
------------
Retrieving Results for Index 248: Yarqā.
Closest hotel to Yarqā at 31.9785 , 35.6947 is بلايستيشن المهندس شاكر.
------------
Retrieving Results for Index 320: Piet Retief.
Closest hotel to Piet Retief at -27.0071 , 30.8132 is RH Private Hospital Piet Retief.
------------
Retrieving Results for Index 386: Kalat.
Closest hotel to Kalat at 29.0225 , 66.5916 is Agha super store.
------------
Retrieving Results for Index 409: Mhlambanyatsi.
Closest hotel to Mhlambanyatsi at -26.45 , 31.0167 is Usuthu Golf Club.
------------
Retrieving Results for Index 499: Safi.
Closest hotel to Safi at 32.2994

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [27]:
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=3)

marker_layer = gmaps.marker_layer(locations, #weights=hum_rate, 
                                 #dissipating=False, max_intensity=100,
                                # point_radius = 1, opacity=1
                               )
#marker_layer = gmaps.marker_layer(locations2, #hover_text='', label='', info_box_content=hotel_info
                                 

fig.add_layer(marker_layer)
fig.add_layer(marker_layer)
plt.savefig("vacationheatmapwithhotels.png")

fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

In [ ]:
# Add marker layer ontop of heat map


# Display figure
